# Data Analysis ETH Transactions

# Data - Accredited Investors

## get data from Google API

In [2]:
print("here I got the source data")

here I got the source data


## explore the data

In [2]:
import pandas as pd
import numpy as np

from tqdm import notebook
notebook.tqdm().pandas()

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
df = pd.DataFrame(values)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

## clean the data

In [ ]:
#only keep observations with a wallet_address
df = df.drop(columns=['number']) #drop empty column
df_NA = df.replace('^\s*$', np.nan, regex=True) #encode empty space as NA
df = df_NA.dropna()
df

In [ ]:
#create a copy of the old df
df2 = df
df2

In [8]:
#get invalid addresses (not long enough)
df2 = df2.astype({'wallet_address': 'string'})
df2["address_length"] = df2.wallet_address.progress_apply(len)
df2["address_length"]=df2["address_length"].astype("category")

In [9]:
#delete invalid addresses
df2 = df2[df2.address_length != 34]

In [11]:
df2['sow_estimated_net_worth_value']= pd.to_numeric(df2['sow_estimated_net_worth_value'])

## feature engineering


### create a dummy for accredited investor

In [14]:
for i in list(range(0, 304)):
    if df2.iloc[i,0]>=1000000: df2.iloc[i,0]= 1
    else: df2.iloc[i,0]= 0

In [15]:
#check how many accredited investors
df2.iloc[:,0].value_counts()

1.0    163
0.0    141
Name: sow_estimated_net_worth_value, dtype: int64

In [17]:
#create a copy of df2
df3 = df2

In [ ]:
#lowered the address string as it is done that way automatically when used in the etherscan 'API'
df3.wallet_address = list(map(str.lower,df2.wallet_address))

In [3]:
#there was an error due to a wrong wallet_address -> it was after "0x58032c2fd021473eeee3c076bb83c3ee68a15064"
print("censored: I filtered that one out")#take a look at these addresses to find the wrong one
#the wrong address (i: 295) had a different format: bc1qnxsyskg99zx3zfl9mr82tumvnzwdlelryk5wvm instead of 0x58......

censored: I filtered that one out


In [21]:
#delecte all "non-ETH" addresses -> they don´t start with 0x
start, stop = 0, 2
df3["address_beginning"]= df3["wallet_address"].str.slice(start, stop)
#use raw strings -> also handles backlashes as normal characters done by r"string" instead of "string"
df3["address_beginning"] = df3["address_beginning"].astype("category")

<ipython-input-21-3bf06a9bd693>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["address_beginning"]= df3["wallet_address"].str.slice(start, stop)
<ipython-input-21-3bf06a9bd693>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["address_beginning"] = df3["address_beginning"].astype("category")


In [22]:
df3.address_beginning.value_counts()

0x    303
Name: address_beginning, dtype: int64

### get normal transaction data

In [23]:
import requests
from decimal import Decimal 
import certifi
from collections import defaultdict
from datetime import datetime

In [24]:
def list_to_string(_list):
        return ",".join(_list) if isinstance(_list, list) else _list

In [25]:
from time import sleep

def api_call(addresses, numbers_transactions, api_key):    
    # url placeholders
    base_url = "https://api.etherscan.io/api"
    url = f"{base_url}?module=account&action=txlist&address={list_to_string(addresses)}&startblock=0&endblock=99999999&page=1&offset={str(numbers_transactions)}&sort=asc&apikey={api_key}"

    try:
        response = requests.get(url, verify=certifi.where())

        #print(response.status_code)
        if response.status_code == 200:
            address_content = response.json()
            
            if address_content.get("message") == "NOTOK":
                return {}

            # print(1) 
            #1 == 1
            #sleep(1) # TODO this is a hack to prevent 'DNS Erros / Connection refused Erros after a ton of request to the api                
            return address_content.get("result")#list of dictionaries
        else:
            return {}
    except Exception as error:
        return {}
        


In [27]:
#get the data per address (hier funktioniert es nur mit einer adresse)
def get_transactions_by_address(address, numbers_transactions, api_key='QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ'):
    data = defaultdict(dict)

    try: # this makes sure the programm is not crashing; even if the api call failes
        result = api_call(address, numbers_transactions, api_key)

    except Exception as error:
        print(error)

        data[0]["hash"] = "" #hash of the transaction
        data[0]["tx_from"] = "" #sender address
        data[0]["tx_to"] = "" #receiver
        data[0]["eth_value"] = 0 #value in eth
        data[0]["out"] = False # dummy - true if the address is sending the money
        data[0]["confirmed"] = False # if the transaction is confirmed
        data[0]["timeStamp"] = datetime.strptime("2000-08-01 12:30", '%Y-%m-%d %H:%M')
        data[0]["nonce"] = 0 #number of confirmed transactions originated from this address
        data[0]["gas"] = 0 #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[0]["will_gas_Price"] = 0 #How much the transaction originator is willing t. pay in Wei per gas unit
        data[0]["gasUsed"] = 0 #How much gas was used at the end
        data[0]["transaction_cost"] = 0
        data[0]["data_transaction"] = ""
        data[0]["isError"] = 0
        data[0]["cumulativeGasUsed"] = 0
        return data

    
    for n, transaction in enumerate(result):
        hashs = transaction.get("hash")
        tx_from = transaction.get("from")
        tx_to = transaction.get("to")
        value = transaction.get("value")
        confirmations = transaction.get("confirmations")
        time = datetime.utcfromtimestamp(int(transaction.get("timeStamp"))).strftime('%Y-%m-%d %H:%M')
        nonce = transaction.get("nonce")
        gas = float(transaction.get("gas"))
        gasPrice = float(transaction.get("gasPrice"))
        gasUsed = transaction.get("gasUsed")
        datatx = transaction.get("input")
        error = transaction.get("isError")
        cumulativeGasUsed = transaction.get("cumulativeGasUsed")
        
        if tx_from == address:
            out = True
        else: 
            out = False

        eth_value = Decimal(value) / Decimal("1000000000000000000") #float test sodass Zahl
        
        gasprice_eth = Decimal(gasPrice) / Decimal("1000000000000000000")
        transaction_cost = Decimal(gasprice_eth) * Decimal(gasUsed)
        if int(confirmations) >=16:
            confirmed = True
        else:
            confirmed = False
        
        data[n]["hash"] = hashs #hash of the transaction
        data[n]["tx_from"] = tx_from #sender address
        data[n]["tx_to"] = tx_to #receiver
        data[n]["eth_value"] = eth_value #value in eth
        data[n]["out"] = out # dummy - true if the address is sending the money
        data[n]["confirmed"] = confirmed # if the transaction is confirmed
        data[n]["timeStamp"] = datetime.strptime(time, '%Y-%m-%d %H:%M')
        data[n]["nonce"] = nonce #number of confirmed transactions originated from this address
        data[n]["gas"] = gas #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[n]["will_gas_Price"] = gasprice_eth #How much the transaction originator is willing t. pay in Wei per gas unit
        data[n]["gasUsed"] = gasUsed #How much gas was used at the end
        data[n]["transaction_cost"] = transaction_cost
        data[n]["data_transaction"] = datatx
        data[n]["isError"] = error
        data[n]["cumulativeGasUsed"] = cumulativeGasUsed
    
    return data    
    #reminder: die Berechnungen müssen für´s Umrechnen in Float übertragen werden

    

In [28]:
#create a data frame that consists out of all the transactions of the whole addresses
new_df = df3.wallet_address.progress_apply(get_transactions_by_address, numbers_transactions = 1500)

In [29]:
def normalize_data(df):
    temp_list = []
    for key, value in df.items():
        for k,v in value.items():
            temp_list.append(v)

    return pd.DataFrame(temp_list)

In [ ]:
new_df

In [ ]:
transaction_df = normalize_data(new_df)
transaction_df # That´s the final edge list of the normal TX

## Get address Balance

In [36]:
def balance_api_call(addresses, api_key):
    # url placeholders
    
    base_url = "https://api.etherscan.io/api"
    url = f"{base_url}?module=account&action=balance&address={list_to_string(addresses)}&tag=latest&apikey={api_key}"

    try:
        response = requests.get(url, verify=certifi.where())

        #print(response.status_code)
        if response.status_code == 200:
            address_content = response.json()
            
            if address_content.get("message") == "NOTOK":
                return {}

            #sleep(1) # TODO this is a hack to prevent 'DNS Erros / Connection refused Erros after a ton of request to the api                
            return address_content.get("result")#list of dictionaries
        else:
            return {}
    except Exception as error:
        print(error)
        return {}

In [37]:
#Get balance for addresses (date: Monday, August 30, 8pm CEST)
address_balance = []

for index, row in df3.iterrows():
    result = balance_api_call(row.wallet_address, "a censored API key")
    address_balance.append({"address_balance":result})

    
address_balance = pd.DataFrame(address_balance)
df3.loc[:,"address_balance"]= address_balance

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [ ]:
df3

## Get the neighboars of the fetched data


In [65]:
def get_first_neighboars(lookup_address, transactions=transaction_df):
    """
        param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    outgoing_transactions =  transactions[(transactions.tx_from == lookup_person) & (transactions.out == True)]
    outgoing_neighboars = outgoing_transactions.drop_duplicates(subset='tx_to')
    # print(len(outgoing_neighboars))

    incomming_transactions =  transactions[(transactions.tx_to == lookup_person) & (transactions.out == False)]
    incomming_neighboars = incomming_transactions.drop_duplicates(subset='tx_from')
    # print(len(incomming_neighboars))

    list_ = outgoing_neighboars.tx_to.to_list()
    list_ += incomming_neighboars.tx_from.to_list()


    first_neighboars = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars = first_neighboars.drop_duplicates(subset='neighboar')

    return first_neighboars



In [66]:
def get_first_out_neighboars(lookup_address, transactions=transaction_df):
    """
        param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    outgoing_transactions =  transactions[(transactions.tx_from == lookup_person) & (transactions.out == True)]
    outgoing_neighboars = outgoing_transactions.drop_duplicates(subset='tx_to')
    # print(len(outgoing_neighboars))

    list_ = outgoing_neighboars.tx_to.to_list()

    first_neighboars_out = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars_out = first_neighboars_out.drop_duplicates(subset='neighboar')

    return first_neighboars_out

In [67]:
def get_first_neighboars_in(lookup_address, transactions=transaction_df):
    """
        param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    incomming_transactions =  transactions[(transactions.tx_to == lookup_person) & (transactions.out == False)]
    incomming_neighboars = incomming_transactions.drop_duplicates(subset='tx_from')
    # print(len(incomming_neighboars))

    list_ = incomming_neighboars.tx_from.to_list()


    first_neighboars_in = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars_in = first_neighboars_in.drop_duplicates(subset='neighboar')

    return first_neighboars_in




## Get first neighboar for every address in d3

In [ ]:
neighboars = []

for index, row in df3.iterrows():
    
    result = get_first_neighboars(row.wallet_address)
    neighboars.append(result)
    
neighboars = pd.concat(neighboars)
neighboars

In [ ]:
neighboars_out = []

for index, row in df3.iterrows():
    
    result = get_first_out_neighboars(row.wallet_address)
    neighboars_out.append(result)
    
neighboars_out = pd.concat(neighboars_out)
neighboars_out

In [ ]:
neighboars_in = []

for index, row in df3.iterrows():
    
    result = get_first_neighboars_in(row.wallet_address)
    neighboars_in.append(result)
    
neighboars_in = pd.concat(neighboars_in)
neighboars_in

## Get transactions for first neighboars

In [ ]:
neighboar_transactions = []

placeholder = []
for i in range(0, len(neighboars)):
    id_ = neighboars.iloc[i].id
    neighboar_ = neighboars.iloc[i].neighboar

    transactions = get_transactions_by_address(neighboar_, numbers_transactions=1600)
    transactions = pd.DataFrame(transactions).transpose()
    transactions["id"] = id_
    transactions["neighboar"] = neighboar_
    placeholder.append(transactions)

first_neighboars_transactions = pd.concat(placeholder)
first_neighboars_transactions



In [ ]:
first_neighboars_transactions.to_csv("first_neighb_transactions.csv")

In [ ]:
EL_neighb_normal = pd.read_csv("first_neighb_transactions.csv", header = 0)
EL_neighb_normal #edge list of neighbor tx (normal)

## Calculate TX of the normal edge lists in USD

### calc USD for TX original adresses

In [7]:
#read in the USD/ETH data
ethUSD= pd.read_csv('ETH_1min.txt', header = None)
ethUSD.columns = ["timeStamp", "Open", "High", "Low", "Close", "Volume"]
ethUSD_s = ethUSD[["timeStamp",  "Close"]].copy()

In [34]:
#redefine the timeStamp column with a string
transaction_df.timeStamp = transaction_df.timeStamp.astype(str)

In [35]:
#merge the exchange and the transaction data with respect to the time Stamp #reminder -> some data older than 2016/03 is lost
df_test = pd.merge(transaction_df, ethUSD_s, on='timeStamp')

In [36]:
#calculate approx. value of transaction parameters in USD
df_test["eth_valueUSD"] = df_test.eth_value.astype("float") * df_test.Close
df_test["transaction_costUSD"] = df_test.transaction_cost.astype("float") * df_test.Close
df_test["will_gas_PriceUSD"] = df_test.will_gas_Price.astype("float") * df_test.Close

In [98]:
df_test.to_csv("df_origin_norm_tx.csv")

In [3]:
df_test= pd.read_csv("df_origin_norm_tx.csv")

### calc USD for 1st neighb normal TX 

In [88]:
EL_neighb_normal.timeStamp = EL_neighb_normal.timeStamp.astype(str)

In [90]:
df_Neighb_normal_USD = pd.merge(EL_neighb_normal, ethUSD_s, on='timeStamp')
df_Neighb_normal_USD["eth_valueUSD"] = df_Neighb_normal_USD.eth_value.astype("float") * df_Neighb_normal_USD.Close
df_Neighb_normal_USD["transaction_costUSD"] = df_Neighb_normal_USD.transaction_cost.astype("float") * df_Neighb_normal_USD.Close
df_Neighb_normal_USD["will_gas_PriceUSD"] = df_Neighb_normal_USD.will_gas_Price.astype("float") * df_Neighb_normal_USD.Close

# Get ERC20 TX

In [48]:
from time import sleep

def erc20_api_call(addresses, numbers_transactions, api_key):
    # url placeholders
    
    base_url = "https://api.etherscan.io/api"
    url = f"{base_url}?module=account&action=tokentx&address={list_to_string(addresses)}&startblock=0&endblock=99999999&page=1&offset={str(numbers_transactions)}&sort=asc&apikey={api_key}"

    try:
        response = requests.get(url, verify=certifi.where())

        #print(response.status_code)
        if response.status_code == 200:
            address_content = response.json()
            
            if address_content.get("message") == "NOTOK":
                return {}

            #sleep(1) # TODO this is a hack to prevent 'DNS Erros / Connection refused Erros after a ton of request to the api                
            return address_content.get("result")#list of dictionaries
        else:
            return {}
    except Exception as error:
        print(error)
        return {}
    

In [50]:
#create tx function for getting erc20 tokens 
def get_erc20_token_by_address(address, numbers_transactions, api_key='QXYD8MMH1JIKKQRSAMUUNM16XEJTI5H7NQ'):
    data = defaultdict(dict)

    try: # this makes sure the programm is not crashing; even if the api call failes
        result = erc20_api_call(address, numbers_transactions, api_key)

    except Exception as error:
        print(error)

        data[0]["hash"] = "" #hash of the transaction
        data[0]["tx_from"] = "" #sender address
        data[0]["tx_to"] = "" #receiver
        data[0]["eth_value"] = 0 #value in eth
        data[0]["out"] = False # dummy - true if the address is sending the money
        data[0]["confirmed"] = False # if the transaction is confirmed
        data[0]["timeStamp"] = datetime.strptime("2000-08-01 12:30", '%Y-%m-%d %H:%M')
        data[0]["nonce"] = 0 #number of confirmed transactions originated from this address
        data[0]["gas"] = 0 #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[0]["will_gas_Price"] = 0 #How much the transaction originator is willing t. pay in Wei per gas unit
        data[0]["gasUsed"] = 0 #How much gas was used at the end
        data[0]["transaction_cost"] = 0
        data[0]["data_transaction"] = ""
        data[0]["isError"] = 0
        data[0]["cumulativeGasUsed"] = 0
        data[0]["tokenName"] = ""
        data[0]["tokenSymbol"] = ""
        data[0]["tokenDecimal"] = 0
        return data

    
    for n, transaction in enumerate(result):
        hashs = transaction.get("hash")
        tx_from = transaction.get("from")
        tx_to = transaction.get("to")
        value = transaction.get("value")
        confirmations = transaction.get("confirmations")
        time = datetime.utcfromtimestamp(int(transaction.get("timeStamp"))).strftime('%Y-%m-%d %H:%M')
        nonce = transaction.get("nonce")
        gas = float(transaction.get("gas"))
        gasPrice = float(transaction.get("gasPrice"))
        gasUsed = transaction.get("gasUsed")
        datatx = transaction.get("input")
        error = transaction.get("isError")
        cumulativeGasUsed = transaction.get("cumulativeGasUsed")
        tokenName = transaction.get("tokenName")
        tokenSymbol = transaction.get("tokenSymbol")
        tokenDecimal = transaction.get("tokenDecimal")
        
        if tx_from == address:
            out = True
        else: 
            out = False

        eth_value = Decimal(value) / Decimal("1000000000000000000") #float test sodass Zahl
        
        gasprice_eth = Decimal(gasPrice) / Decimal("1000000000000000000")
        transaction_cost = Decimal(gasprice_eth) * Decimal(gasUsed)
        if int(confirmations) >=16:
            confirmed = True
        else:
            confirmed = False
        
        data[n]["hash"] = hashs #hash of the transaction
        data[n]["tx_from"] = tx_from #sender address
        data[n]["tx_to"] = tx_to #receiver
        data[n]["eth_value"] = eth_value #value in eth
        data[n]["out"] = out # dummy - true if the address is sending the money
        data[n]["confirmed"] = confirmed # if the transaction is confirmed
        data[n]["timeStamp"] = datetime.strptime(time, '%Y-%m-%d %H:%M')
        data[n]["nonce"] = nonce #number of confirmed transactions originated from this address
        data[n]["gas"] = gas #gas Limit -> wie viel gas einheiten man max. zahlen würde
        data[n]["will_gas_Price"] = gasprice_eth #How much the transaction originator is willing t. pay in Wei per gas unit
        data[n]["gasUsed"] = gasUsed #How much gas was used at the end
        data[n]["transaction_cost"] = transaction_cost
        data[n]["data_transaction"] = datatx
        data[n]["isError"] = error
        data[n]["cumulativeGasUsed"] = cumulativeGasUsed
        data[n]["tokenName"] = tokenName
        data[n]["tokenSymbol"] = tokenSymbol
        data[n]["tokenDecimal"] = tokenDecimal
    
    return data    
    #reminder: die Berechnungen müssen für´s Umrechnen in Float übertragen werden

In [51]:
erc20_df = df3.wallet_address.progress_apply(get_erc20_token_by_address, numbers_transactions = 1500)

In [52]:
erc20_df = normalize_data(erc20_df) #edge list of erc20 transaction of the given addreses

In [99]:
erc20_df.to_csv("erc20_normal_tx.csv")

In [ ]:
import pandas as pd
erc20_df = pd.read_csv("erc20_normal_tx.csv")

## Get first neighboar of ERC20 Transaktions

In [53]:
def get_first_ercneighboars(lookup_address, transactions=erc20_df):
    """
        #param: transactions -> pd.Dataframe same shape as transaction_df
    """

    lookup_person = lookup_address
    # print(lookup_person)

    outgoing_transactions =  transactions[(transactions.tx_from == lookup_person) & (transactions.out == True)]
    outgoing_neighboars = outgoing_transactions.drop_duplicates(subset='tx_to')
    # print(len(outgoing_neighboars))

    incomming_transactions =  transactions[(transactions.tx_to == lookup_person) & (transactions.out == False)]
    incomming_neighboars = incomming_transactions.drop_duplicates(subset='tx_from')
    # print(len(incomming_neighboars))

    list_ = outgoing_neighboars.tx_to.to_list()
    list_ += incomming_neighboars.tx_from.to_list()


    first_neighboars = pd.DataFrame({'neighboar': list_, 'id': [lookup_address for i in list_]})
    first_neighboars = first_neighboars.drop_duplicates(subset='neighboar')

    return first_neighboars




In [ ]:
erc20_neighboars = []

for index, row in df3.iterrows():
    
    result = get_first_ercneighboars(row.wallet_address, transactions=erc20_df)
    erc20_neighboars.append(result)
    
erc20_neighboars = pd.concat(erc20_neighboars)
erc20_neighboars

## get tx of first neighb of ERC20 tx

In [ ]:
erc_neighboar_transactions = []

erc_placeholder = []
for i in range(0, len(erc20_neighboars)):
    id_ = erc20_neighboars.iloc[i].id
    erc_neighboar_ = erc20_neighboars.iloc[i].neighboar

    transactions = get_erc20_token_by_address(erc_neighboar_, numbers_transactions=1600)
    transactions = pd.DataFrame(transactions).transpose()
    transactions["id"] = id_
    transactions["neighboar"] = erc_neighboar_
    erc_placeholder.append(transactions)

erc_first_neighboars_transactions = pd.concat(erc_placeholder)
erc_first_neighboars_transactions.iloc[] #edgelist of neighb ERC20 tx




In [ ]:
erc_first_neighboars_transactions.to_csv("erc_first_neighboars_transactions.csv")



In [184]:
EL_ERCneighb_normal = pd.read_csv("erc_first_neighboars_transactions.csv", header = 0)
EL_ERCneighb_normal.head()
random.seed(30)
EL_ERCneib_small = EL_ERCneighb_normal.sample(frac = 0.1) #edge list of neighbor tx (ERC20)

## calc USD value of ERC TX and ERC TX of first neighb

### USD value ERC TX original

In [8]:
erc20_df.timeStamp = erc20_df.timeStamp.astype(str)
df_origin_erc20_USD = pd.merge(erc20_df, ethUSD_s, on='timeStamp')
df_origin_erc20_USD["eth_valueUSD"] = df_origin_erc20_USD.eth_value.astype("float") * df_origin_erc20_USD.Close
df_origin_erc20_USD["transaction_costUSD"] = df_origin_erc20_USD.transaction_cost.astype("float") * df_origin_erc20_USD.Close
df_origin_erc20_USD["will_gas_PriceUSD"] = df_origin_erc20_USD.will_gas_Price.astype("float") * df_origin_erc20_USD.Close

### USD value ERC TX neighb

In [97]:
EL_ERCneighb_normal.timeStamp = EL_ERCneighb_normal.timeStamp.astype(str)
EL_ERCneighb_USD = pd.merge(EL_ERCneighb_normal, ethUSD_s, on='timeStamp')
EL_ERCneighb_USD["eth_valueUSD"] = EL_ERCneighb_USD.eth_value.astype("float") * EL_ERCneighb_USD.Close
EL_ERCneighb_USD["transaction_costUSD"] = EL_ERCneighb_USD.transaction_cost.astype("float") * EL_ERCneighb_USD.Close
EL_ERCneighb_USD["will_gas_PriceUSD"] = EL_ERCneighb_USD.will_gas_Price.astype("float") * EL_ERCneighb_USD.Close

In [185]:
import random
random.seed(30)
EL_ERCneibUSD_small = EL_ERCneighb_USD.sample(frac = 0.1)

# Start Variable creation 



## Variables regarding the "normal transactions" of the original addresses

In [37]:
#Number ingoing transactions
length2=[]
for address in df3.wallet_address:
    length2.append(len(df_test[df_test["tx_to"] == str(address)]))
df3.loc[:,"num_in_trans"]= length2

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [102]:
#Number ingoing transactions
length2=[]
for address in df3.wallet_address:
    length2.append(len(df_test[df_test["tx_to"] == str(address)]))
df3.loc[:,"num_in_trans"]= length2

In [38]:
#Number outgoing transactions
length1=[]
for address in df3.wallet_address:
    length1.append(len(df_test[df_test["tx_from"] == str(address)]))
df3.loc[:,"num_out_trans"]= length1

In [104]:
#average amount outgoing transactions
avg_trans_out=[]
for address in df3.wallet_address:
    avg_trans_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["eth_valueUSD"]].mean())
df3.loc[:,"avg_trans_out"]= avg_trans_out

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [40]:
df3.to_csv("corrected_numoutsins.csv") #this is whee the correction happened

In [105]:
#average amount ingoing transactions
avg_trans_in=[]
for address in df3.wallet_address:
    avg_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].mean())
df3.loc[:,"avg_trans_in"]= avg_trans_in

In [106]:
#sum amount ingoing transactions
sum_trans_in=[]
for address in df3.wallet_address:
    sum_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].sum())
df3.loc[:,"sum_trans_in"]= sum_trans_in

In [107]:
#sum amount outgoing transactions
sum_trans_out=[]
for address in df3.wallet_address:
    sum_trans_out.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].sum())
df3.loc[:,"sum_trans_out"]= sum_trans_out

In [108]:
#max amount outgoing transaction
max_trans_out=[]
for address in df3.wallet_address:
    max_trans_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["eth_valueUSD"]].max())
df3.loc[:,"max_trans_out"]= max_trans_out

In [109]:
#max amount ingoing transaction
max_trans_in=[]
for address in df3.wallet_address:
    max_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].max())
df3.loc[:,"max_trans_in"]= max_trans_in

In [110]:
#standard deviation amount ingoing transaction
std_trans_in=[]
for address in df3.wallet_address:
    std_trans_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["eth_valueUSD"]].std())
df3.loc[:,"std_trans_in"]= std_trans_in

In [111]:
#standard deviation amount out going transaction
std_trans_out=[]
for address in df3.wallet_address:
    std_trans_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["eth_valueUSD"]].std())
df3.loc[:,"std_trans_out"]= std_trans_out

In [112]:
#min amount outgoing transaction
min_trans_out=[]
for address in df3.wallet_address:
    min_trans_out.append(df_test.loc[(df_test['tx_from'] == address) & (df_test["eth_valueUSD"]!=0),["eth_valueUSD"]].min())
df3.loc[:,"min_trans_out"]= min_trans_out

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [113]:
#min amount ingoing transaction
min_trans_in=[]
for address in df3.wallet_address:
    min_trans_in.append(df_test.loc[(df_test['tx_to'] == address) & (df_test["eth_valueUSD"]!=0),["eth_valueUSD"]].min())
df3.loc[:,"min_trans_in"]= min_trans_in

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [114]:
#number of unique outgoing transaction partners 
unique_tpartn_out=[]
for address in df3.wallet_address:
    unique_tpartn_out.append(df_test.loc[(df_test['tx_from'] == address), ["tx_to"]].nunique())
df3.loc[:,"unique_tpartn_out"]= unique_tpartn_out


In [115]:
#number of unique ingoing transaction partners 
unique_tpartn_in=[]
for address in df3.wallet_address:
    unique_tpartn_in.append(df_test.loc[(df_test['tx_to'] == address), ["tx_from"]].nunique())
df3.loc[:,"unique_tpartn_in"]= unique_tpartn_in

In [116]:
#average outg. transaction costs
avg_transcost_out=[]
for address in df3.wallet_address:
    avg_transcost_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"avg_transcost_out"]= avg_transcost_out 

In [117]:
#average inc. transaction costs
avg_transcost_in=[]
for address in df3.wallet_address:
    avg_transcost_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"avg_transcost_in"]= avg_transcost_in

In [118]:
#max transaction costs outgoing
max_transcost_out=[]
for address in df3.wallet_address:
    max_transcost_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["transaction_costUSD"]].max())
df3.loc[:,"max_transcost_out"]= max_transcost_out

In [119]:
#max transaction costs ingoing
max_transcost_in=[]
for address in df3.wallet_address:
    max_transcost_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["transaction_costUSD"]].max())
df3.loc[:,"max_transcost_in"]= max_transcost_in

In [120]:
#standard deviation transaction costs  out going 
std_transcost_out=[]
for address in df3.wallet_address:
    std_transcost_out.append(df_test.loc[lambda df_test: df_test['tx_from'] == str(address), ["transaction_costUSD"]].std())
df3.loc[:,"std_transcost_out"]= std_transcost_out

In [121]:
#standard deviation transaction costs  ingoing 
std_transcost_in=[]
for address in df3.wallet_address:
    std_transcost_in.append(df_test.loc[lambda df_test: df_test['tx_to'] == str(address), ["transaction_costUSD"]].std())
df3.loc[:,"std_transcost_in"]= std_transcost_in

In [122]:
#avg. amount of hours between outg transactions
timedelta_trans_out=[]
for address in df3.wallet_address:
    t = pd.to_datetime(df_test.loc[lambda df_test: df_test['tx_from'] == address, ["timeStamp"]].timeStamp)
    time_diff = [j-i for i, j in zip(t[:-1], t[1:])]
    time_list = []
    for delta in time_diff:
        time_list.append(delta / pd.Timedelta('1 hour'))
    timedelta_trans_out.append(pd.Series(time_list).mean())
    
df3.loc[:,"timedelta_trans_out"]= timedelta_trans_out


<ipython-input-122-62aaec5a58bd>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  timedelta_trans_out.append(pd.Series(time_list).mean())
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [123]:
#avg. amount of hours between in transactions
timedelta_trans_in=[]
for address in df3.wallet_address:
    t = pd.to_datetime(df_test.loc[lambda df_test: df_test['tx_to'] == address, ["timeStamp"]].timeStamp)
    time_diff = [j-i for i, j in zip(t[:-1], t[1:])]
    time_list = []
    for delta in time_diff:
        time_list.append(delta / pd.Timedelta('1 hour'))
    timedelta_trans_in.append(pd.Series(time_list).mean())
    
df3.loc[:,"timedelta_trans_in"]= timedelta_trans_in

<ipython-input-123-d986470869da>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  timedelta_trans_in.append(pd.Series(time_list).mean())


In [124]:
#amount of hours between the first and last outgoing transaction
duration_out=[]
for address in df3.wallet_address:
    t = pd.to_datetime(df_test.loc[lambda df_test: df_test['tx_from'] == address, ["timeStamp"]].timeStamp)
    time_diff = [j-i for i, j in zip(t[:1], t[-1:])]
    time_list = []
    for delta in time_diff:
        time_list.append(delta / pd.Timedelta('1 hour'))
    duration_out.append(pd.Series(time_list).mean())
    
df3.loc[:,"duration_out"]= duration_out

<ipython-input-124-2cfee261268d>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  duration_out.append(pd.Series(time_list).mean())


In [125]:
#avg Number ingoing transactions per neighb
length_neighb=[]
for address in df3.wallet_address:
    result = 0
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars.loc[(neighboars["id"] == str(address))]
    length_p_neighb = []
    for index, row in filtered_neighb.iterrows():
        length_p_neighb.append(len(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar)]))
        result = pd.DataFrame(length_p_neighb)
        result = result.iloc[:,0].mean()
    length_neighb.append(result)
        
df3.loc[:,"num_in_trans_1neighb"]= length_neighb



/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [126]:
#percentage of transaction partners who received only one transaction from the address (sog. one Night Stands
avg_oneNightStands_out = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        p_address = filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["neighboar"]].count()
        result_p_address.append(p_address)
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].eq(1).sum(axis = 0)
        result2 = result
        counts = filtered_neighb.iloc[:,1].count()
        result3 = result2/counts
    avg_oneNightStands_out.append(result3)
        
df3.loc[:,"avg_oneNightStands_out"]= avg_oneNightStands_out

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [127]:
#percentage of transaction partners who received only one transaction from the address (sog. one Night Stands
avg_oneNightStands_in = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        p_address = filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["neighboar"]].count()
        result_p_address.append(p_address)
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].eq(1).sum(axis = 0)
        result2 = result
        counts = filtered_neighb.iloc[:,1].count()
        result3 = result2/counts
    avg_oneNightStands_in.append(result3)
        
df3.loc[:,"avg_oneNightStands_in"]= avg_oneNightStands_in

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
filtered_neighb.iloc[:,1].count()

## variables regarding the normal transactions of the first neighbs 

In [ ]:
#avg Number ingoing transactions per neighb
length_neighb=[]
for address in df3.wallet_address:
    result = 0
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    length_p_neighb = []
    for index, row in filtered_neighb.iterrows():
        length_p_neighb.append(len(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar)]))
        result = pd.DataFrame(length_p_neighb)
        result = result.iloc[:,0].mean()
    length_neighb.append(result)
        
df3.loc[:,"num_in_trans_1neighb"]= length_neighb



In [ ]:
#avg Number outgoing transactions per neighb
length_neighb_out=[]
for address in df3.wallet_address:
    result = 0
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    length_p_neighb = []
    for index, row in filtered_neighb.iterrows():
        length_p_neighb.append(len(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar)]))
        result = pd.DataFrame(length_p_neighb)
        result = result.iloc[:,0].mean()
    length_neighb_out.append(result)
        
df3.loc[:,"num_out_trans_1neighb"]= length_neighb_out



In [ ]:
#average amount outgoing transactions p neighb
avg_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_tx_out_neighb.append(result)
        
df3.loc[:,"avg_tx_out_neighb"]= avg_tx_out_neighb

In [ ]:
#average amount ingoing transactions p neighb
avg_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_tx_in_neighb.append(result)
        
df3.loc[:,"avg_tx_in_neighb"]= avg_tx_in_neighb

In [131]:
#max amount outgoing transactions 
max_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_tx_out_neighb.append(result)
        
df3.loc[:,"max_tx_out_neighb"]= max_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [132]:
#max amount ingoing transactions 
max_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_tx_in_neighb.append(result)
        
df3.loc[:,"max_tx_in_neighb"]= max_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [133]:
#sum amount ingoing transactions 
sum_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].sum())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    sum_tx_in_neighb.append(result)
        
df3.loc[:,"sum_tx_in_neighb"]= sum_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [134]:
#sum amount outgoing transactions 
sum_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].sum())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    sum_tx_out_neighb.append(result)
        
df3.loc[:,"sum_tx_out_neighb"]= sum_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [135]:
#standard deviation amount outgoing transaction
std_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].std())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    std_tx_out_neighb.append(result)
        
df3.loc[:,"std_tx_out_neighb"]= std_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [136]:
#standard deviation amount ingoing transaction
std_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["eth_valueUSD"]].std())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    std_tx_in_neighb.append(result)
        
df3.loc[:,"std_tx_in_neighb"]= std_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [137]:
#number of unique outgoing transaction partners 
unique_tx_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["tx_to"]].nunique().iloc[0])
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    unique_tx_out_neighb.append(result)
        
df3.loc[:,"unique_tx_out_neighb"]= unique_tx_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [138]:
#number of unique ingoing transaction partners 
unique_tx_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["tx_to"]].nunique().iloc[0])
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    unique_tx_in_neighb.append(result)
        
df3.loc[:,"unique_tx_in_neighb"]= unique_tx_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [139]:
#average amount outgoing transaction cost 
avg_txcost_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_txcost_out_neighb.append(result)
        
df3.loc[:,"avg_txcost_out_neighb"]= avg_txcost_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [140]:
#average amount ingoing transaction cost 
avg_txcost_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].mean())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    avg_txcost_in_neighb.append(result)
        
df3.loc[:,"avg_txcost_in_neighb"]= avg_txcost_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [141]:
#max amount ingoing transaction cost
max_txcost_in_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == False)]
    filtered_neighb = neighboars_in.loc[(neighboars_in["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_txcost_in_neighb.append(result)
        
df3.loc[:,"max_txcost_in_neighb"]= max_txcost_in_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [142]:
#max amount out going transaction cost
max_txcost_out_neighb = []
for address in df3.wallet_address:
    result = 0
    result_p_address = []
    filtered_df = df_Neighb_normal_USD.loc[(df_Neighb_normal_USD["id"] == str(address))&(df_Neighb_normal_USD["out"] == True)]
    filtered_neighb = neighboars_out.loc[(neighboars_out["id"] == str(address))]
    for index, row in filtered_neighb.iterrows():
        result_p_address.append(filtered_df.loc[filtered_df["neighboar"] == str(row.neighboar), ["transaction_costUSD"]].max())
        result = pd.DataFrame(result_p_address)
        result = result.iloc[:,0].mean()
    max_txcost_out_neighb.append(result)
        
df3.loc[:,"max_txcost_out_neighb"]= max_txcost_out_neighb

/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Kenneth/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [149]:
df3.to_csv("first_data.csv")



## Get ERC variables for origin addresses

In [4]:
df3 = pd.read_csv("first_data.csv")

In [11]:
df_origin_erc20_USD['tokenSymbol'] = pd.Categorical(df_origin_erc20_USD['tokenSymbol'])

In [12]:
print (df_origin_erc20_USD['tokenSymbol'].cat.categories)

Index(['$ANRX', '$BASED', '$BREAST', '$DG', '$FDD', '$ITG', '$JINDOINU',
       '0xBTC', '0xBUTT', '1-UP',
       ...
       'yvWETH', 'yvYFI', 'yveCRV-DAO', 'yyDAI+yUSDC+yUSDT+yTUSD', 'zSK',
       'ziot', 'Ͼ', '⚗️', '✌(◕‿-)✌  tinyurl.com/nicetokens', '✨'],
      dtype='object', length=2128)


In [13]:
#Number ingoing transactions
lengthERC=[]
for address in df3.wallet_address:
    lengthERC.append(len(df_origin_erc20_USD[df_origin_erc20_USD["tx_to"] == str(address)]))
df3.loc[:,"num_in_transERC"]= lengthERC

In [14]:
#Number ingoing transactions
lengthERC=[]
for address in df3.wallet_address:
    lengthERC.append(len(df_origin_erc20_USD[df_origin_erc20_USD["tx_from"] == str(address)]))
df3.loc[:,"num_out_transERC"]= lengthERC

In [25]:
#number of unique outgoing transaction partners 
ERCunique_tpartn_out=[]
for address in df3.wallet_address:
    ERCunique_tpartn_out.append(df_origin_erc20_USD.loc[(df_origin_erc20_USD['tx_from'] == address), ["tx_to"]].nunique())
df3.loc[:,"ERCunique_tpartn_out"]= ERCunique_tpartn_out

In [26]:
#number of unique ingoing transaction partners 
ERCunique_tpartn_in=[]
for address in df3.wallet_address:
    ERCunique_tpartn_in.append(df_origin_erc20_USD.loc[(df_origin_erc20_USD['tx_to'] == address), ["tx_from"]].nunique())
df3.loc[:,"ERCunique_tpartn_in"]= ERCunique_tpartn_in

In [27]:
#average outg. transaction costs
ERCavg_transcost_out=[]
for address in df3.wallet_address:
    ERCavg_transcost_out.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_from'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"ERCavg_transcost_out"]= ERCavg_transcost_out 

In [68]:
#average inc. transaction costs
ERCavg_transcost_in=[]
for address in df3.wallet_address:
    ERCavg_transcost_in.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_to'] == str(address), ["transaction_costUSD"]].mean())
df3.loc[:,"ERCavg_transcost_in"]= ERCavg_transcost_in

In [69]:
#max transaction costs outgoing
ERCmax_transcost_out=[]
for address in df3.wallet_address:
    ERCmax_transcost_out.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_from'] == str(address), ["transaction_costUSD"]].max())
df3.loc[:,"ERCmax_transcost_out"]= ERCmax_transcost_out

In [108]:
#number of different traded tokens (outgoing + ingoing)
ERC_nb_tkn_out=[]
ERC_nb_tkn_in=[]
ERC_nb_tkn_tot=[]
for address in df3.wallet_address:
    ERC_nb_tkn_out.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_from'] == str(address), ["tokenSymbol"]].nunique().iloc[0])
    ERC_nb_tkn_in.append(df_origin_erc20_USD.loc[lambda df_origin_erc20_USD: df_origin_erc20_USD['tx_to'] == str(address), ["tokenSymbol"]].nunique().iloc[0])
ERC_nb_tkn_tot = pd.DataFrame(ERC_nb_tkn_out)+ pd.DataFrame(ERC_nb_tkn_in)
#df3.loc[:,"ERC_nb_tkn_tot"]= ERC_nb_tkn_tot

In [98]:
df3 = df3.drop(['ERC_nb_tkn_tot'], axis=1)

In [109]:
ERC_nb_tkn_tot = ERC_nb_tkn_tot.rename({0: 'ERC_nb_tkn_tot'}, axis = 1)

## Herding Variable Creation

In [196]:
#number of sells
hot_sell_per = df_test[df_test["out"]==True].resample("H", on = "timeStamp").out.count()
hot_sell_per

timeStamp
2017-05-01 21:00:00     1
2017-05-01 22:00:00     0
2017-05-01 23:00:00     0
2017-05-02 00:00:00     0
2017-05-02 01:00:00     0
                       ..
2021-07-16 19:00:00     5
2021-07-16 20:00:00     3
2021-07-16 21:00:00    12
2021-07-16 22:00:00    10
2021-07-16 23:00:00    17
Freq: H, Name: out, Length: 36891, dtype: int64

In [244]:
#number of buys
hot_buy_per = df_test[df_test["out"]==False].resample("D", on = "timeStamp").hash.count()
hot_buy_per

timeStamp
2017-05-01     3
2017-05-02     0
2017-05-03     0
2017-05-04     0
2017-05-05     0
              ..
2021-07-12     7
2021-07-13     0
2021-07-14    11
2021-07-15    12
2021-07-16    11
Freq: D, Name: hash, Length: 1538, dtype: int64

In [245]:
pressure_periods = pd.concat([hot_buy_per, hot_sell_per], axis = 1, join = "inner")
pressure_periods = pressure_periods.rename({"hash":"no_buys", "out":"no_sells"}, axis = "columns")
pressure_periods["p"]= pressure_periods["no_buys"] - pressure_periods["no_sells"]
pressure_periods[["p"]].describe()

,p
count,1537.000000
mean,3.409239
std,25.193465
min,-16.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,877.000000


In [248]:
sorted_pp = pressure_periods.sort_values(by=["p"])
sorted_pp.head(10)


,no_buys,no_sells,p
timeStamp,,,
2021-04-22,30,46,-16
2020-11-14,0,14,-14
2020-10-29,6,19,-13
2020-11-16,2,13,-11
2020-10-24,5,16,-11
2018-05-05,0,10,-10
2020-12-26,4,13,-9
2020-11-20,5,14,-9
2020-12-10,4,12,-8


In [319]:
sellpump = sorted_pp.head(10)
sellpump

,no_buys,no_sells,p
timeStamp,,,
2021-04-22,30,46,-16
2020-11-14,0,14,-14
2020-10-29,6,19,-13
2020-11-16,2,13,-11
2020-10-24,5,16,-11
2018-05-05,0,10,-10
2020-12-26,4,13,-9
2020-11-20,5,14,-9
2020-12-10,4,12,-8


In [338]:
buypump = sorted_pp.tail(10)
buypump

,no_buys,no_sells,p
timeStamp,,,
2021-05-16,49,4,45
2021-04-27,52,6,46
2021-05-20,58,6,52
2021-05-03,53,1,52
2021-04-03,58,2,56
2021-04-01,144,15,129
2021-03-31,140,9,131
2021-03-30,197,9,188
2021-04-02,294,6,288


In [335]:
high_pressure_sells =[]

for address in df3.wallet_address:
    test = pd.DataFrame()
    eth_bough = []
    address_id = []
    time_index = []
    filter_df = df_test.loc[df_test["tx_from"]==str(address)]
    for i in filter_df.resample("D", on = "timeStamp").eth_value.sum().index:
        if i in sellpump.index: 
            filter2 = filter_df[filter_df["timeStamp"]== str(i)]
            address_id.append(address)
            time_index.append(i)
    test["address_id"] = address_id
    test["time_index"] = time_index
    high_pressure_sells.append(len(test))
df3.loc[:,"high_pressure_sells"]= high_pressure_sells
        #buys.append()

In [339]:
high_pressure_buys =[]

for address in df3.wallet_address:
    test = pd.DataFrame()
    eth_bough = []
    address_id = []
    time_index = []
    filter_df = df_test.loc[df_test["tx_from"]==str(address)]
    for i in filter_df.resample("D", on = "timeStamp").eth_value.sum().index:
        if i in buypump.index: 
            filter2 = filter_df[filter_df["timeStamp"]== str(i)]
            address_id.append(address)
            time_index.append(i)
    test["address_id"] = address_id
    test["time_index"] = time_index
    high_pressure_buys.append(len(test))
df3.loc[:,"high_pressure_buys"]= high_pressure_buys
        #buys.append()

In [341]:
df3.to_csv("second_data1709.csv")

## Get EL

In [150]:
df_test
normal_trans_ELy = df_test[["tx_from", "tx_to"]]
normal_trans_ELy["weight"] = df_test["eth_valueUSD"]+df_test["transaction_costUSD"]

<ipython-input-150-5c196cef28d0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_trans_ELy["weight"] = df_test["eth_valueUSD"]+df_test["transaction_costUSD"]


In [ ]:
normal_trans_ELy

In [ ]:
EL_ERCneighb_USD.head()

In [187]:
EL_ERCneighb_USD.head()
finEL_ERCneighb_USD = EL_ERCneibUSD_small[["tx_from", "tx_to"]]#take only 10%
finEL_ERCneighb_USD["weight"] = EL_ERCneibUSD_small["eth_valueUSD"]+EL_ERCneibUSD_small["transaction_costUSD"]


<ipython-input-187-e9849771caee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finEL_ERCneighb_USD["weight"] = EL_ERCneibUSD_small["eth_valueUSD"]+EL_ERCneibUSD_small["transaction_costUSD"]


In [153]:
df_origin_erc20_USD.head()
EL_origin_erc20y = df_origin_erc20_USD[["tx_from", "tx_to"]]
EL_origin_erc20y["weight"] = df_origin_erc20_USD["eth_valueUSD"]+df_origin_erc20_USD["transaction_costUSD"]

<ipython-input-153-3a6d3bbca05f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EL_origin_erc20y["weight"] = df_origin_erc20_USD["eth_valueUSD"]+df_origin_erc20_USD["transaction_costUSD"]


In [189]:
small_Neighb_normal_USD.head()
EL_Neigh_norm = small_Neighb_normal_USD[["tx_from", "tx_to"]]
EL_Neigh_norm["weight"] = small_Neighb_normal_USD["eth_valueUSD"]+small_Neighb_normal_USD["transaction_costUSD"]


<ipython-input-189-098846bb639c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EL_Neigh_norm["weight"] = small_Neighb_normal_USD["eth_valueUSD"]+small_Neighb_normal_USD["transaction_costUSD"]


In [190]:
EL_final = normal_trans_ELy.append([EL_Neigh_norm, EL_origin_erc20y, finEL_ERCneighb_USD], ignore_index = True)

In [193]:
EL_only_origin = normal_trans_ELy.append(EL_origin_erc20y, ignore_index = True)

In [196]:
EL_only_origin.isnull().sum()

tx_from    0
tx_to      0
weight     0
dtype: int64

In [200]:
EL_only_origin.to_csv("EL_only_origin.csv")